# SimCLR
PyTorch implementation of SimCLR: A Simple Framework for Contrastive Learning of Visual Representations by T. Chen et al. With support for the LARS (Layer-wise Adaptive Rate Scaling) optimizer.

[Link to paper](https://arxiv.org/pdf/2002.05709.pdf)


## Setup the repository

In [1]:
# !git clone https://github.com/spijkervet/SimCLR.git
# %cd SimCLR
# # !wget https://github.com/Spijkervet/SimCLR/releases/download/1.2/checkpoint_100.tar
# !sh setup.sh || python3 -m pip install -r requirements.txt || exit 1
# !pip install  pyyaml --upgrade

In [2]:
# !pip install pretrainedmodels

# # !git clone https://github.com/spijkervet/SimCLR.git
# %cd SimCLR
# # !wget https://github.com/Spijkervet/SimCLR/releases/download/1.2/checkpoint_100.tar
# !sh setup.sh || python3 -m pip install -r requirements.txt || exit 1
# !pip install  pyyaml --upgrade

In [3]:
# # !git clone https://github.com/Kabongosalomon/SimCLR-1.git
%cd SimCLR-1
# # !wget https://github.com/Spijkervet/SimCLR/releases/download/1.2/checkpoint_100.tar
# !sh setup.sh || python3 -m pip install -r requirements.txt || exit 1
# !pip install  pyyaml --upgrade

/root/cassava_disease_classification/salomon_exp/SimCLR-1


In [4]:
!pip install ipdb
import ipdb

!pip install optuna

import torch
import torchvision
import torchvision.transforms as transforms
import argparse

# from experiment import ex
from model import load_model
from utils import post_config_hook

from modules import LogisticRegression

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import glob
import numpy as np

from torchvision import transforms, datasets, models


from PIL import Image

# Part 1:
## SimCLR pre-training

In [5]:
# whether to use a TPU or not (set in Runtime -> Change Runtime Type)
use_tpu = False

#### Install PyTorch/XLA

In [6]:
if use_tpu:
    VERSION = "20200220" #@param ["20200220","nightly", "xrt==1.15.0"]
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version $VERSION

In [7]:
import os
import torch

if use_tpu:
    # imports the torch_xla package for TPU support
    import torch_xla
    import torch_xla.core.xla_model as xm
    dev = xm.xla_device()
    print(dev)

import torchvision
import argparse

from torch.utils.tensorboard import SummaryWriter

apex = False
try:
    from apex import amp
    apex = True
except ImportError:
    print(
        "Install the apex package from https://www.github.com/nvidia/apex to use fp16 for training"
    )

from model import load_model, save_model
from modules import NT_Xent
from modules.transformations import TransformsSimCLR
from utils import post_config_hook

Install the apex package from https://www.github.com/nvidia/apex to use fp16 for training


### Load arguments from `config/config.yaml`

In [8]:
from pprint import pprint
from utils.yaml_config_hook import yaml_config_hook

config = yaml_config_hook("./config/config.yaml")
args = argparse.Namespace(**config)

if use_tpu:
    args.device = dev
else:
    args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

args.out_dir = "logs"
if not os.path.exists("logs"):
    os.makedirs("logs")

In [9]:
pprint(vars(args))

{'batch_size': 128,
 'dataset': 'CIFAR10',
 'device': device(type='cuda', index=0),
 'epoch_num': 100,
 'epochs': 100,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 256,
 'logistic_epochs': 500,
 'model_path': 'logs/0',
 'normalize': True,
 'optimizer': 'Adam',
 'out_dir': 'logs',
 'pretrain': True,
 'projection_dim': 64,
 'resnet': 'resnet50',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [10]:
64*2

128

In [11]:
# ### override any configuration parameters here, e.g. to adjust for use on GPUs on the Colab platform:
# args.batch_size = 64
# args.resnet = "resnet18" # se_resnet50, resnet18, se_resnext101_32x4d, resnet50
# pprint(vars(args))

In [12]:
args.dataset = 'casava'

In [13]:
# args.epochs = 100
# args.epoch_num = 100

# args.projection_dim = 64

# args.logistic_epochs = 100

# args.optimizer = "LARS" #"Adam"

### Load dataset into train loader

In [14]:
data_path = "../data/train/train"
test_path = "../data/test/test"
extraimage_path = "../data/extraimages/extraimages"

In [15]:
print('Train set:')
class_distrbution = {}
for cls in os.listdir(data_path):
    print('{}:{}'.format(cls, len(os.listdir(os.path.join(data_path, cls)))))
    class_distrbution[cls] =  len(os.listdir(os.path.join(data_path, cls)))
im = Image.open(data_path+'/cgm/train-cgm-738.jpg')
print(im.size)
class_distrbution

Train set:
cgm:773
cbb:466
cmd:2658
cbsd:1443
healthy:316
(500, 500)


{'cbb': 466, 'cbsd': 1443, 'cgm': 773, 'cmd': 2658, 'healthy': 316}

In [16]:
mean=[0.4543, 0.5137, 0.3240]
std=[0.1949, 0.1977, 0.1661]

class CassavaDataset(Dataset):
    def __init__(self, path, size, s=1, mutation = False):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.mutation = mutation
        color_jitter = torchvision.transforms.ColorJitter(
            0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s
        )
        
        self.train_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.RandomResizedCrop(size),
                torchvision.transforms.RandomHorizontalFlip(),  # with 0.5 probability
                torchvision.transforms.RandomApply([color_jitter], p=0.8),
                torchvision.transforms.RandomGrayscale(p=0.2),
                torchvision.transforms.ToTensor(),
            ]
        )
        
        self.test_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize((size, size)),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(mean=mean,std=std)
            ]
        )
        

        files = []
        class_names = {}
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])

                name = str(i)+'-'+className
                if name not in class_names:
                    class_names[name] = 1
                else:
                    class_names[name] += 1
        self.file_list = files
        files = None
    
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        image = Image.open(fileName)

        if self.mutation:
            image1 = self.train_transform(image)
            image2 = self.train_transform(image)
            
            sample = [[image1, image2], classCategory]
        else:
            
            image = self.test_transform(image)
            sample = [image, classCategory]

        return sample

In [16]:
size = 448

train_data = CassavaDataset(data_path, size, s=1, mutation = False)

test_data = CassavaDataset(test_path, size, s=1, mutation = False)

extraimage_data = CassavaDataset(extraimage_path, size, s=1, mutation = True)

#######################################################################
validation_split = 0.2
shuffle_dataset = True
# random_seed= 42 #42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
#     np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]
########################################################################

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

batch_size_train = args.batch_size# 125
batch_size_eval = args.batch_size#250
n_workers = 2

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size_train,
                                             sampler = train_sampler, num_workers = n_workers)

valid_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size_eval,
                                             sampler = valid_sampler, num_workers = n_workers)

# unlabeled_loader = torch.utils.data.DataLoader(extraimage_data, batch_size = batch_size_eval, 
#                                               shuffle =shuffle_dataset, num_workers = n_workers)

In [13]:
# # number of subprocesses to use for data loading
# num_workers = 6
# # how many samples per batch to load
# batch_size = 125

# train_data = datasets.ImageFolder(data_path, transform=train_transforms)
# test_data = datasets.ImageFolder(test_path, transform=test_transforms)
# extraimage_data = datasets.ImageFolder(extraimage_path, transform=train_transforms)

# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
#                                            num_workers=num_workers, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
#                                           num_workers=num_workers)

# unlabeled_loader = torch.utils.data.DataLoader(extraimage_data, batch_size=batch_size,
#                                                num_workers=num_workers) # to make batch_size work, I had to moove all the unlabeled data in a 0 folder

In [13]:
# next(unlabeled_loader.__iter__())

In [14]:
# len(next(unlabeled_loader.__iter__())[1])

In [15]:
# root = "./datasets"

# train_sampler = None

# if args.dataset == "STL10":
#     train_dataset = torchvision.datasets.STL10(
#         root, split="unlabeled", download=True, transform=TransformsSimCLR(size=96)
#     )
# elif args.dataset == "CIFAR10":
#     train_dataset = torchvision.datasets.CIFAR10(
#         root, download=True, transform=TransformsSimCLR(size=32)
#     )
# else:
#     raise NotImplementedError

# train_loader = torch.utils.data.DataLoader(
#     train_dataset,
#     batch_size=args.batch_size,
#     shuffle=(train_sampler is None),
#     drop_last=True,
#     num_workers=args.workers,
#     sampler=train_sampler,
# )

### Load the SimCLR model, optimizer and learning rate scheduler

In [20]:
pprint(vars(args))

{'batch_size': 64,
 'dataset': 'casava',
 'device': device(type='cuda', index=0),
 'epoch_num': 100,
 'epochs': 100,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 256,
 'logistic_epochs': 500,
 'model_path': 'logs/0',
 'normalize': True,
 'optimizer': 'Adam',
 'out_dir': 'logs',
 'pretrain': True,
 'projection_dim': 64,
 'resnet': 'resnet18',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [42]:
# pprint(vars(args))

In [22]:
# args.projection_dim = 125

In [24]:
model, optimizer, scheduler = load_model(args, unlabeled_loader, reload_model=False)

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnet50-ce0d4300.pth" to /root/.cache/torch/checkpoints/se_resnet50-ce0d4300.pth


Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /root/.cache/torch/checkpoints/se_resnext101_32x4d-3b2fe3d8.pth


In [25]:
model

SimCLR(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [26]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0002
    weight_decay: 0
)

In [27]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0002
    weight_decay: 0
)

In [28]:
scheduler

### Setup TensorBoard for logging experiments

In [29]:
tb_dir = os.path.join(args.out_dir, "colab")
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
writer = SummaryWriter(log_dir=tb_dir)

### Create the mask that will remove correlated samples from the negative examples

### Initialize the criterion (NT-Xent loss)

In [30]:
args.batch_size

64

In [31]:
criterion = NT_Xent(args.batch_size, args.temperature, args.device)

In [32]:
criterion

NT_Xent(
  (criterion): CrossEntropyLoss()
  (similarity_f): CosineSimilarity()
)

### Start training

In [33]:
def train(args, train_loader, model, criterion, optimizer, writer):
    loss_epoch = 0
    for step, ((x_i, x_j), _) in enumerate(train_loader):

        if x_i.shape[0] != args.batch_size:
            continue
        optimizer.zero_grad()
        x_i = x_i.to(args.device)
        x_j = x_j.to(args.device)

        # positive pair, with encoding
        h_i, z_i = model(x_i)
        h_j, z_j = model(x_j)
        
        #h = self.encoder(x)
        #z = self.projector(h)

#         ipdb.set_trace()
        loss = criterion(z_i, z_j)

        if apex and args.fp16:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()

        optimizer.step()

        if step % 50 == 0:
            print(f"Step [{step}/{len(train_loader)}]\t Loss: {loss.item()}")

        writer.add_scalar("Loss/train_epoch", loss.item(), args.global_step)
        loss_epoch += loss.item()
        args.global_step += 1

    return loss_epoch

In [34]:
import pdb, traceback, sys

if __name__ == '__main__':
    try:
        
        args.global_step = 0
        args.current_epoch = 0
        for epoch in range(args.start_epoch, args.epochs):
            lr = optimizer.param_groups[0]['lr']
        #     ipdb.set_trace()
            loss_epoch = train(args, unlabeled_loader, model, criterion, optimizer, writer)

            if scheduler:
                scheduler.step()

            if epoch % 100 == 0:
                save_model(args, model, optimizer)

            writer.add_scalar("Loss/train", loss_epoch / len(unlabeled_loader), epoch)
            writer.add_scalar("Misc/learning_rate", lr, epoch)
            print(
                f"Epoch [{epoch}/{args.epochs}]\t Loss: {loss_epoch / len(unlabeled_loader)}\t lr: {round(lr, 5)}"
            )
            args.current_epoch += 1

        ## end training
        save_model(args, model, optimizer)
    except:
        extype, value, tb = sys.exc_info()
        traceback.print_exc()
        pdb.post_mortem(tb)

Step [0/197]	 Loss: 4.831396102905273
Step [50/197]	 Loss: 4.74738883972168
Step [100/197]	 Loss: 4.656131267547607
Step [150/197]	 Loss: 4.717471122741699
Epoch [0/100]	 Loss: 4.653891478698266	 lr: 0.0002
Step [0/197]	 Loss: 4.50970458984375
Step [50/197]	 Loss: 4.5008721351623535
Step [100/197]	 Loss: 4.5789875984191895
Step [150/197]	 Loss: 4.267772674560547
Epoch [1/100]	 Loss: 4.360394860282162	 lr: 0.0002
Step [0/197]	 Loss: 4.344832897186279
Step [50/197]	 Loss: 4.176105976104736
Step [100/197]	 Loss: 4.2121806144714355
Step [150/197]	 Loss: 4.231607437133789
Epoch [2/100]	 Loss: 4.078589260275594	 lr: 0.0002
Step [0/197]	 Loss: 3.914879083633423
Step [50/197]	 Loss: 4.05787467956543
Step [100/197]	 Loss: 4.022071361541748
Step [150/197]	 Loss: 4.030446529388428
Epoch [3/100]	 Loss: 3.89950888290018	 lr: 0.0002
Step [0/197]	 Loss: 3.766413927078247
Step [50/197]	 Loss: 3.8228940963745117
Step [100/197]	 Loss: 3.826023578643799
Step [150/197]	 Loss: 3.6736721992492676
Epoch [4/1

In [35]:
save_model(args, model, optimizer)

In [14]:
# import pdb, traceback, sys

# def bombs():
#     a = []
#     print (a[0])

# if __name__ == '__main__':
#     try:
#         bombs()
#     except:
#         extype, value, tb = sys.exc_info()
#         traceback.print_exc()
#         pdb.post_mortem(tb)

## Download last checkpoint to local drive (replace `100` with `args.epochs`)

In [14]:
# from google.colab import files
# files.download('./logs/checkpoint_'+args.epochs.tar) # checkpoint_100.tar

# Part 2:
## Linear evaluation using logistic regression, using weights from frozen, pre-trained SimCLR model

In [23]:
def train(args, loader, simclr_model, model, criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    for step, (x, y) in enumerate(loader):
        optimizer.zero_grad()

        x = x.to(args.device)
        y = y.to(args.device)

        # get encoding
        with torch.no_grad():
            h, z = simclr_model(x)
            # h = 512
            # z = 64
            # h = self.encoder(x)
            # z = self.projector(h)

        output = model(h)
        loss = criterion(output, y)

        predicted = output.argmax(1)
        acc = (predicted == y).sum().item() / y.size(0)
        accuracy_epoch += acc

        loss.backward()
        optimizer.step()

        loss_epoch += loss.item()
        if step % 100 == 0:
            print(f"Step [{step}/{len(loader)}]\t Loss: {loss.item()}\t Accuracy: {acc}")

    return loss_epoch, accuracy_epoch

In [18]:
def test(args, loader, simclr_model, model, criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    model.eval()
    for step, (x, y) in enumerate(loader):
        model.zero_grad()

        x = x.to(args.device)
        y = y.to(args.device)

        # get encoding
        with torch.no_grad():
            h, z = simclr_model(x)
            # h = 512
            # z = 64
            # h = self.encoder(x)
            # z = self.projector(h)

        output = model(z)
        loss = criterion(output, y)

        predicted = output.argmax(1)
        acc = (predicted == y).sum().item() / y.size(0)
        accuracy_epoch += acc

        loss_epoch += loss.item()


    return loss_epoch, accuracy_epoch

In [19]:
from pprint import pprint
from utils.yaml_config_hook import yaml_config_hook

config = yaml_config_hook("./config/config.yaml")
pprint(config)
args = argparse.Namespace(**config)

if use_tpu:
    args.device = dev
else:
    args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

{'batch_size': 128,
 'dataset': 'CIFAR10',
 'epoch_num': 100,
 'epochs': 100,
 'fp16': False,
 'fp16_opt_level': 'O2',
 'logistic_batch_size': 256,
 'logistic_epochs': 500,
 'model_path': 'logs/0',
 'normalize': True,
 'optimizer': 'Adam',
 'pretrain': True,
 'projection_dim': 64,
 'resnet': 'resnet50',
 'seed': 42,
 'start_epoch': 0,
 'temperature': 0.5,
 'weight_decay': 1e-06,
 'workers': 16}


In [20]:
# args.batch_size = 32
# args.resnet = "se_resnext101_32x4d" # 
# args.model_path = "logs"
# args.epoch_num = 20 # 100
# args.projection_dim = 32

# args.logistic_epochs = 100

### Load dataset into train/test dataloaders

In [21]:
# root = "./datasets"
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# if args.dataset == "STL10":
#     train_dataset = torchvision.datasets.STL10(
#         root, split="train", download=True, transform=torchvision.transforms.ToTensor()
#     )
#     test_dataset = torchvision.datasets.STL10(
#         root, split="test", download=True, transform=torchvision.transforms.ToTensor()
#     )
# elif args.dataset == "CIFAR10":
#     train_dataset = torchvision.datasets.CIFAR10(
#         root, train=True, download=True, transform=transform
#     )
#     test_dataset = torchvision.datasets.CIFAR10(
#         root, train=False, download=True, transform=transform
#     )
# else:
#     raise NotImplementedError

# train_loader = torch.utils.data.DataLoader(
#     train_dataset,
#     batch_size=args.logistic_batch_size,
#     shuffle=True,
#     drop_last=True,
#     num_workers=args.workers,
# )

# test_loader = torch.utils.data.DataLoader(
#     test_dataset,
#     batch_size=args.logistic_batch_size,
#     shuffle=False,
#     drop_last=True,
#     num_workers=args.workers,
# )

### Load SimCLR model and load model weights

In [43]:
args.resnet = "resnet18" # se_resnet50, resnet18, se_resnext101_32x4d, resnet50


In [22]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 40,
                                             sampler = train_sampler, num_workers = n_workers)

valid_loader = torch.utils.data.DataLoader(train_data, batch_size = 40,
                                             sampler = valid_sampler, num_workers = n_workers)

In [25]:
!ls

LICENSE      environment.yml  media		run_all.sh     tsne_on_h.png
README.md    experiment.py    model.py		setup.sh       tsne_on_z.png
__pycache__  logs	      modules		setup_apex.sh  utils
config	     main.py	      requirements.txt	testing


In [32]:
args.model_path

'logs/0'

In [44]:
args.model_path = 'logs'

In [45]:
simclr_model, _, _ = load_model(args, train_loader, reload_model=True)
simclr_model = simclr_model.to(args.device)
simclr_model.eval()

SimCLR(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [46]:
args

Namespace(batch_size=64, dataset='casava', device=device(type='cuda', index=0), epoch_num=100, epochs=100, fp16=False, fp16_opt_level='O2', logistic_batch_size=256, logistic_epochs=500, model_path='logs', normalize=True, optimizer='LARS', out_dir='logs', pretrain=True, projection_dim=64, resnet='resnet18', seed=42, start_epoch=0, temperature=0.5, weight_decay=1e-06, workers=16)

In [47]:
args.logistic_epochs = 100

In [ ]:
# ## Logistic Regression
# n_classes = 5 # stl-10
# model = LogisticRegression(simclr_model.n_features, n_classes)
# model = model.to(args.device)

In [32]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
criterion = torch.nn.CrossEntropyLoss()

In [33]:
args.logistic_epochs

100

In [34]:
# best_acc = 0.80
# for epoch in range(args.logistic_epochs):
#     loss_epoch, accuracy_epoch = train(args, train_loader, simclr_model, model, criterion, optimizer)
#     print(f"Epoch [{epoch}/{args.logistic_epochs}]\t Loss: {loss_epoch / len(train_loader)}\t Accuracy: {accuracy_epoch / len(train_loader)}")
    
#     # final testing
#     loss_epoch, accuracy_epoch = test(args, valid_loader, simclr_model, model, criterion, optimizer)
#     print(f"[FINAL]\t Loss: {loss_epoch / len(valid_loader)}\t Accuracy: {accuracy_epoch / len(valid_loader)}")
#     if accuracy_epoch / len(valid_loader) > best_acc :  
#         print(f"Model {valid_loader}_{accuracy_epoch / len(valid_loader)} saved")
#         save_model(args, simclr_model, optimizer)

# # final testing
# loss_epoch, accuracy_epoch = test(args, valid_loader, simclr_model, model, criterion, optimizer)
# print(f"[FINAL]\t Loss: {loss_epoch / len(valid_loader)}\t Accuracy: {accuracy_epoch / len(valid_loader)}")

## Using model my trained model

### Load SimCLR model and load model weights

In [54]:
def objective(trial):
    
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
#     model = model = torch.hub.load('pytorch/vision:v0.5.0', 'resnet18', pretrained=True)

    args.resnet = "resnet18" 
    args.model_path = 'logs'
    
    simclr_model, _, _ = load_model(args, train_loader, reload_model=True)
    simclr_model = simclr_model.to(args.device)
    simclr_model.eval()   
    
    n_classes = 5 # stl-10
    
    model = LogisticRegression(64, n_classes)
    model = model.to(args.device)
    
    lr  = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    
    
    
#     criterion = nn.CrossEntropyLoss(weight = weights)
    args.logistic_epochs = 10
#     args.resnet = "resnet18" 
    
    for epoch in range(args.logistic_epochs):
        loss_epoch, accuracy_epoch = train(args, train_loader, simclr_model, model, criterion, optimizer)
        print(f"Epoch [{epoch}/{args.logistic_epochs}]\t Loss: {loss_epoch / len(train_loader)}\t Accuracy: {accuracy_epoch / len(train_loader)}")

    # final testing
    loss_epoch_val, accuracy_epoch_val = test(args, valid_loader, simclr_model, model, criterion, optimizer)
    print(f"[FINAL]\t Loss: {loss_epoch_val / len(valid_loader)}\t Accuracy: {accuracy_epoch_val / len(valid_loader)}")
            
     # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
            
            
    return (accuracy_epoch_val / len(valid_loader))

In [55]:
def train(args, loader, simclr_model, model, criterion, optimizer):
    loss_epoch = 0
    accuracy_epoch = 0
    for step, (x, y) in enumerate(loader):
        optimizer.zero_grad()

        x = x.to(args.device)
        y = y.to(args.device)

        # get encoding
        with torch.no_grad():
            h, z = simclr_model(x)
            # h = 512
            # z = 64
            # h = self.encoder(x)
            # z = self.projector(h)

        output = model(z)
        loss = criterion(output, y)

        predicted = output.argmax(1)
        acc = (predicted == y).sum().item() / y.size(0)
        accuracy_epoch += acc

        loss.backward()
        optimizer.step()

        loss_epoch += loss.item()
        if step % 100 == 0:
            print(f"Step [{step}/{len(loader)}]\t Loss: {loss.item()}\t Accuracy: {acc}")

    return loss_epoch, accuracy_epoch

In [56]:
import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective, n_trials=5)

Step [0/36]	 Loss: 1.5597878694534302	 Accuracy: 0.5078125
Epoch [0/10]	 Loss: 1.5654357704851363	 Accuracy: 0.4626157407407407
Step [0/36]	 Loss: 1.5543488264083862	 Accuracy: 0.4765625
Epoch [1/10]	 Loss: 1.5546982023451064	 Accuracy: 0.4673080632716049
Step [0/36]	 Loss: 1.5475690364837646	 Accuracy: 0.484375
Epoch [2/10]	 Loss: 1.5425658259126875	 Accuracy: 0.4704282407407407
Step [0/36]	 Loss: 1.523428201675415	 Accuracy: 0.5390625
Epoch [3/10]	 Loss: 1.5298074351416693	 Accuracy: 0.4688271604938271
Step [0/36]	 Loss: 1.5295724868774414	 Accuracy: 0.4609375
Epoch [4/10]	 Loss: 1.515473657184177	 Accuracy: 0.47122878086419756
Step [0/36]	 Loss: 1.4940001964569092	 Accuracy: 0.5078125
Epoch [5/10]	 Loss: 1.4999496671888564	 Accuracy: 0.47082851080246907
Step [0/36]	 Loss: 1.5320987701416016	 Accuracy: 0.3671875
Epoch [6/10]	 Loss: 1.484392782052358	 Accuracy: 0.4700279706790123
Step [0/36]	 Loss: 1.4794028997421265	 Accuracy: 0.4609375
Epoch [7/10]	 Loss: 1.4688045978546143	 Accurac

[I 2020-05-10 16:30:37,658] Finished trial#0 with value: 0.4673789589823468 with parameters: {'lr': 1.0268907214912501e-05}. Best is trial#0 with value: 0.4673789589823468.


Step [0/36]	 Loss: 1.5890510082244873	 Accuracy: 0.3515625
Epoch [0/10]	 Loss: 1.251209106710222	 Accuracy: 0.5228105709876544
Step [0/36]	 Loss: 1.2028460502624512	 Accuracy: 0.5625
Epoch [1/10]	 Loss: 1.1441894753111734	 Accuracy: 0.5669801311728395
Step [0/36]	 Loss: 1.0272876024246216	 Accuracy: 0.671875
Epoch [2/10]	 Loss: 1.1268040090799332	 Accuracy: 0.572974537037037
Step [0/36]	 Loss: 0.9981377720832825	 Accuracy: 0.6484375
Epoch [3/10]	 Loss: 1.1056363234917324	 Accuracy: 0.5812548225308642
Step [0/36]	 Loss: 1.072443962097168	 Accuracy: 0.5703125
Epoch [4/10]	 Loss: 1.090991112920973	 Accuracy: 0.5927710262345679
Step [0/36]	 Loss: 1.0818747282028198	 Accuracy: 0.5703125
Epoch [5/10]	 Loss: 1.0834623393085268	 Accuracy: 0.5938753858024691
Step [0/36]	 Loss: 0.9466356039047241	 Accuracy: 0.6640625
Epoch [6/10]	 Loss: 1.0649780382712681	 Accuracy: 0.600535300925926
Step [0/36]	 Loss: 1.0486745834350586	 Accuracy: 0.59375
Epoch [7/10]	 Loss: 1.0546617988083098	 Accuracy: 0.6047

[I 2020-05-10 16:37:13,254] Finished trial#1 with value: 0.6026090342679127 with parameters: {'lr': 0.0033413692566497443}. Best is trial#1 with value: 0.6026090342679127.


Step [0/36]	 Loss: 1.5895073413848877	 Accuracy: 0.453125
Epoch [0/10]	 Loss: 1.5149625937143962	 Accuracy: 0.46715856481481477
Step [0/36]	 Loss: 1.4617310762405396	 Accuracy: 0.421875
Epoch [1/10]	 Loss: 1.3746660384866927	 Accuracy: 0.4700279706790123
Step [0/36]	 Loss: 1.4152501821517944	 Accuracy: 0.375
Epoch [2/10]	 Loss: 1.3153442243734996	 Accuracy: 0.4720293209876544
Step [0/36]	 Loss: 1.3149607181549072	 Accuracy: 0.4609375
Epoch [3/10]	 Loss: 1.2867509656482272	 Accuracy: 0.4728153935185185
Step [0/36]	 Loss: 1.2565126419067383	 Accuracy: 0.46875
Epoch [4/10]	 Loss: 1.250851270225313	 Accuracy: 0.511058063271605
Step [0/36]	 Loss: 1.2592544555664062	 Accuracy: 0.515625
Epoch [5/10]	 Loss: 1.2117097742027707	 Accuracy: 0.5469810956790124
Step [0/36]	 Loss: 1.1422476768493652	 Accuracy: 0.609375
Epoch [6/10]	 Loss: 1.1882009969817267	 Accuracy: 0.5589023919753087
Step [0/36]	 Loss: 1.1197820901870728	 Accuracy: 0.5546875
Epoch [7/10]	 Loss: 1.1670451760292053	 Accuracy: 0.5642

[I 2020-05-10 16:43:46,653] Finished trial#2 with value: 0.5767052829698858 with parameters: {'lr': 0.00011379177522615647}. Best is trial#1 with value: 0.6026090342679127.


Step [0/36]	 Loss: 1.614890217781067	 Accuracy: 0.1328125
Epoch [0/10]	 Loss: 1.60211577018102	 Accuracy: 0.2480565200617284
Step [0/36]	 Loss: 1.589152216911316	 Accuracy: 0.3671875
Epoch [1/10]	 Loss: 1.575769864850574	 Accuracy: 0.4674238040123456
Step [0/36]	 Loss: 1.555338740348816	 Accuracy: 0.5
Epoch [2/10]	 Loss: 1.546531852748659	 Accuracy: 0.4702112268518518
Step [0/36]	 Loss: 1.5245764255523682	 Accuracy: 0.5
Epoch [3/10]	 Loss: 1.5133103165361617	 Accuracy: 0.4696277006172839
Step [0/36]	 Loss: 1.4855536222457886	 Accuracy: 0.484375
Epoch [4/10]	 Loss: 1.4763211376137204	 Accuracy: 0.4692274305555555
Step [0/36]	 Loss: 1.4497541189193726	 Accuracy: 0.53125
Epoch [5/10]	 Loss: 1.4384895198875003	 Accuracy: 0.47122878086419756
Step [0/36]	 Loss: 1.4050042629241943	 Accuracy: 0.5078125
Epoch [6/10]	 Loss: 1.407761851946513	 Accuracy: 0.4688271604938271
Step [0/36]	 Loss: 1.381539225578308	 Accuracy: 0.4609375
Epoch [7/10]	 Loss: 1.3780446118778653	 Accuracy: 0.4712287808641975

[I 2020-05-10 16:50:23,830] Finished trial#3 with value: 0.46652712876427826 with parameters: {'lr': 2.2826254631909797e-05}. Best is trial#1 with value: 0.6026090342679127.


Step [0/36]	 Loss: 1.632752776145935	 Accuracy: 0.09375


KeyboardInterrupt: 

In [25]:
size = 448

train_data = CassavaDataset(data_path, size, s=1, mutation = False)

test_data = CassavaDataset(test_path, size, s=1, mutation = False)

extraimage_data = CassavaDataset(extraimage_path, size, s=1, mutation = True)

#######################################################################
validation_split = 0.0
shuffle_dataset = True
# random_seed= 42 #42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
#     np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]
########################################################################

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

batch_size_train = args.batch_size# 125
batch_size_eval = args.batch_size#250
n_workers = 2

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size_train,
                                             sampler = train_sampler, num_workers = n_workers)

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
#     model = model = torch.hub.load('pytorch/vision:v0.5.0', 'resnet18', pretrained=True)

args.resnet = "resnet18" 
args.model_path = 'logs'

simclr_model, _, _ = load_model(args, train_loader, reload_model=True)
simclr_model = simclr_model.to(args.device)
simclr_model.eval()   

n_classes = 5 # stl-10

model = LogisticRegression(simclr_model.n_features, n_classes)
model = model.to(args.device)

lr  = 0.000979019687620664 #trial.suggest_loguniform('lr', 1e-5, 1e-1)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()



#     criterion = nn.CrossEntropyLoss(weight = weights)
args.logistic_epochs = 100
#     args.resnet = "resnet18" 

for epoch in range(args.logistic_epochs):
    loss_epoch, accuracy_epoch = train(args, train_loader, simclr_model, model, criterion, optimizer)
    print(f"Epoch [{epoch}/{args.logistic_epochs}]\t Loss: {loss_epoch / len(train_loader)}\t Accuracy: {accuracy_epoch / len(train_loader)}")

# final testing
# loss_epoch_val, accuracy_epoch_val = test(args, valid_loader, simclr_model, model, criterion, optimizer)
# print(f"[FINAL]\t Loss: {loss_epoch_val / len(valid_loader)}\t Accuracy: {accuracy_epoch_val / len(valid_loader)}")

Step [0/45]	 Loss: 1.6216480731964111	 Accuracy: 0.15625
Epoch [0/100]	 Loss: 1.2076541662216187	 Accuracy: 0.5449652777777778
Step [0/45]	 Loss: 1.11709725856781	 Accuracy: 0.546875
Epoch [1/100]	 Loss: 1.0561380452579923	 Accuracy: 0.6009837962962963
Step [0/45]	 Loss: 0.9637601971626282	 Accuracy: 0.6640625
Epoch [2/100]	 Loss: 0.9941215303209093	 Accuracy: 0.6255208333333333
Step [0/45]	 Loss: 1.0275079011917114	 Accuracy: 0.59375
Epoch [3/100]	 Loss: 0.9609492195977105	 Accuracy: 0.6366898148148148
Step [0/45]	 Loss: 0.9461528062820435	 Accuracy: 0.6796875
Epoch [4/100]	 Loss: 0.9621600641144646	 Accuracy: 0.6425347222222222
Step [0/45]	 Loss: 0.815538763999939	 Accuracy: 0.6796875
Epoch [5/100]	 Loss: 0.9265490889549255	 Accuracy: 0.6528935185185185
Step [0/45]	 Loss: 0.898435652256012	 Accuracy: 0.671875
Epoch [6/100]	 Loss: 0.9259289158715143	 Accuracy: 0.6500578703703704
Step [0/45]	 Loss: 0.9151264429092407	 Accuracy: 0.65625
Epoch [7/100]	 Loss: 0.9125212960773044	 Accuracy:

# Load a pretrained Model

We will need to rename the model saved on path, to allow the model load it instaed of our own model

In [36]:
def objective(trial):
    
    from pprint import pprint
    from utils.yaml_config_hook import yaml_config_hook

    config = yaml_config_hook("./config/config.yaml")
#     pprint(config)
    args = argparse.Namespace(**config)

    if use_tpu:
        args.device = dev
    else:
        args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
#     model = model = torch.hub.load('pytorch/vision:v0.5.0', 'resnet18', pretrained=True)

    args.model_path = 'logs'
    
    simclr_model, _, _ = load_model(args, train_loader, reload_model=True)
    simclr_model = simclr_model.to(args.device)
    simclr_model.eval()   
    
    n_classes = 5 # stl-10
    
    model = LogisticRegression(64, n_classes)
    model = model.to(args.device)
    
    lr  = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    
    
    
#     criterion = nn.CrossEntropyLoss(weight = weights)
    args.logistic_epochs = 10
#     args.resnet = "resnet18" 
    
    for epoch in range(args.logistic_epochs):
        loss_epoch, accuracy_epoch = train(args, train_loader, simclr_model, model, criterion, optimizer)
        print(f"Epoch [{epoch}/{args.logistic_epochs}]\t Loss: {loss_epoch / len(train_loader)}\t Accuracy: {accuracy_epoch / len(train_loader)}")

    # final testing
    loss_epoch_val, accuracy_epoch_val = test(args, valid_loader, simclr_model, model, criterion, optimizer)
    print(f"[FINAL]\t Loss: {loss_epoch_val / len(valid_loader)}\t Accuracy: {accuracy_epoch_val / len(valid_loader)}")
            
     # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
            
            
    return (accuracy_epoch_val / len(valid_loader))

In [44]:
simclr_model, _, _ = load_model(args, train_loader, reload_model=True)
simclr_model = simclr_model.to(args.device)
simclr_model.eval() 

model = LogisticRegression(64, 5)
model = model.to(args.device)
model

LogisticRegression(
  (model): Linear(in_features=64, out_features=5, bias=True)
  (fc_1): Linear(in_features=64, out_features=700, bias=True)
  (fc_2): Linear(in_features=700, out_features=200, bias=True)
  (fc_out): Linear(in_features=200, out_features=5, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [47]:
h, z = simclr_model(next(train_loader.__iter__())[0].to(args.device))

In [48]:
h.shape

torch.Size([128, 512])

In [51]:
model(z).shape

torch.Size([128, 5])

In [38]:
import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective, n_trials=50)

[W 2020-05-10 16:15:19,018] Setting status of trial#0 as TrialState.FAIL because of the following error: RuntimeError('Error(s) in loading state_dict for SimCLR:\n\tMissing key(s) in state_dict: "encoder.layer1.0.conv3.weight", "encoder.layer1.0.bn3.weight", "encoder.layer1.0.bn3.bias", "encoder.layer1.0.bn3.running_mean", "encoder.layer1.0.bn3.running_var", "encoder.layer1.0.downsample.0.weight", "encoder.layer1.0.downsample.1.weight", "encoder.layer1.0.downsample.1.bias", "encoder.layer1.0.downsample.1.running_mean", "encoder.layer1.0.downsample.1.running_var", "encoder.layer1.1.conv3.weight", "encoder.layer1.1.bn3.weight", "encoder.layer1.1.bn3.bias", "encoder.layer1.1.bn3.running_mean", "encoder.layer1.1.bn3.running_var", "encoder.layer1.2.conv1.weight", "encoder.layer1.2.bn1.weight", "encoder.layer1.2.bn1.bias", "encoder.layer1.2.bn1.running_mean", "encoder.layer1.2.bn1.running_var", "encoder.layer1.2.conv2.weight", "encoder.layer1.2.bn2.weight", "encoder.layer1.2.bn2.bias", "enco

RuntimeError: Error(s) in loading state_dict for SimCLR:
	Missing key(s) in state_dict: "encoder.layer1.0.conv3.weight", "encoder.layer1.0.bn3.weight", "encoder.layer1.0.bn3.bias", "encoder.layer1.0.bn3.running_mean", "encoder.layer1.0.bn3.running_var", "encoder.layer1.0.downsample.0.weight", "encoder.layer1.0.downsample.1.weight", "encoder.layer1.0.downsample.1.bias", "encoder.layer1.0.downsample.1.running_mean", "encoder.layer1.0.downsample.1.running_var", "encoder.layer1.1.conv3.weight", "encoder.layer1.1.bn3.weight", "encoder.layer1.1.bn3.bias", "encoder.layer1.1.bn3.running_mean", "encoder.layer1.1.bn3.running_var", "encoder.layer1.2.conv1.weight", "encoder.layer1.2.bn1.weight", "encoder.layer1.2.bn1.bias", "encoder.layer1.2.bn1.running_mean", "encoder.layer1.2.bn1.running_var", "encoder.layer1.2.conv2.weight", "encoder.layer1.2.bn2.weight", "encoder.layer1.2.bn2.bias", "encoder.layer1.2.bn2.running_mean", "encoder.layer1.2.bn2.running_var", "encoder.layer1.2.conv3.weight", "encod

# TSNE

In [32]:
train_loader.dataset.classes

['cgm', 'cbb', 'cmd', 'cbsd', 'healthy']

In [33]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
tsne = TSNE()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [34]:
# train_loader = torch.utils.data.DataLoader(train_data, batch_size = 50,
#                                              sampler = train_sampler, num_workers = n_workers)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
len(train_loader.dataset.classes) == len(['green','orange','brown','dodgerblue','red'])

True

In [41]:
# a function used to plot t-SNE visualizations
def plot_vecs_n_labels(v,labels,fname):
    fig = plt.figure(figsize = (10, 10))
    plt.axis('off')
    sns.set_style("darkgrid")
    sns.scatterplot(v[:,0], v[:,1], hue=labels, legend='full', palette=['green','orange','brown','dodgerblue','red'])
    plt.legend(train_loader.dataset.classes)
    plt.savefig(fname)
    plt.close()

# Boolean variable to control whether to perform t-SNE visualization or not
TSNEVIS = True
# 
if TSNEVIS:
    # run resnet in eval mode
    simclr_model.eval()

    # get TSNE visualizations of training dataset
    for (i, (input, output)) in enumerate(train_loader):
        x = input
        x = x.to(device)
        y = simclr_model(x)[0] #h, z
        y_tsne = tsne.fit_transform(y.cpu().data)
        labels = output
        plot_vecs_n_labels(y_tsne,labels,'tsne_on_z'+str(i)+'.png')
        x = None
        y = None
        y_tsne = None
        sample_batched = None
        if i==5:
            break
#         break
            
    # get TSNE visualizations of training dataset
    for (i, (input, output)) in enumerate(train_loader):
        x = input
        x = x.to(device)
        y = simclr_model(x)[0] #h, z
        y_tsne = tsne.fit_transform(y.cpu().data)
        labels = output
        plot_vecs_n_labels(y_tsne,labels,'tsne_on_h'+str(i)+'.png')
        x = None
        y = None
        y_tsne = None
        sample_batched = None
        break
        
        if i==5:
            break

RuntimeError: CUDA out of memory. Tried to allocate 490.00 MiB (GPU 0; 15.90 GiB total capacity; 14.90 GiB already allocated; 165.81 MiB free; 15.04 GiB reserved in total by PyTorch)

# Submission

In [27]:
train_loader.dataset.classes

['cgm', 'cbb', 'cmd', 'cbsd', 'healthy']

In [28]:
class_names = {i:label for i, label in enumerate(train_loader.dataset.classes)}
class_names

{0: 'cgm', 1: 'cbb', 2: 'cmd', 3: 'cbsd', 4: 'healthy'}

In [29]:
from os import listdir, makedirs, getcwd, remove


In [30]:
def process_image(image_dir):
    # Process a PIL image for use in a PyTorch model
    # tensor.numpy().transpose(1, 2, 0)
    image = Image.open(image_dir)
#     preprocess = torchvision.transforms.Compose(
#             [
#                 torchvision.transforms.Resize((224, 224)),
#                 torchvision.transforms.CenterCrop(224),
#                 torchvision.transforms.ToTensor()
#             ]
#         )
    
    preprocess = transforms.Compose([ transforms.Resize(224),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                      transforms.Normalize(mean=mean,std=std)])
    image = preprocess(image)
    # Convert 2D image to 1D vector
    image = np.expand_dims(image, 0)
    image = torch.from_numpy(image)
    inputs = image.to(device)
    return inputs

In [31]:
# Using our model to predict the label
def predict(image1, simclr_model, model):
    # Pass the image through our model
    output_hat = simclr_model(image1)
    output = model(output_hat[0])
    # Reverse the log function in our output
    output = torch.exp(output)
    # Get the top predicted class, and the output percentage for
    # that class
    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

In [32]:
test_directory = "../data/test/test/0"
predictions, test_image_fileName = [], []
try:
    test_images = listdir(test_directory)
    for images in test_images:
        test_image_fileName.append(images)
        image = process_image(f'{test_directory}/{images}')
        top_prob, top_class = predict(image, simclr_model, model)
        predictions.append(class_names[top_class])
except Exception as e:
    print(e)

In [65]:
import pandas as pd

In [66]:
print("[INFO] Creating pandas dataframe")
submission_data = {"Category":predictions,"Id":test_image_fileName,}
submission_data_frame = pd.DataFrame(submission_data)

[INFO] Creating pandas dataframe


In [67]:
submission_data_frame.head()

,Category,Id
0,healthy,test-img-1567.jpg
1,cgm,test-img-2833.jpg
2,healthy,test-img-1833.jpg
3,healthy,test-img-1916.jpg
4,cmd,test-img-1641.jpg


In [69]:
submission_data_frame.to_csv('submission'+'_SimCLR.csv', header=True, index=False)